<a href="https://colab.research.google.com/github/SPS-L/pyeplan/blob/master/examples/SHS_Planning_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

# An illustrative investment planning example to design an energy solution for a single household

---
<center>

## Introduction

This example has been prepared in form of a report. It provides a step-by-step guideline on how to use PyEPLAN to decide on the necessary generators that need to be installed to cover the electricity load consumption of a single household.


<center>

![Single Household](https://raw.githubusercontent.com/SPS-L/pyeplan/master/examples/1_bus.png)

<center>




## Instructions

*   This example is designed to be run on Google Colab. If this is the first time you use Colab, please take a 3-minute break and check out [this video](https://youtu.be/inN8seMm7UI).

*   To get a quickstart, simply select "**Runtime->Run all**" in the menu and accept the prompt. This will execute the entire notebook and you can read through it like a report.

*   At the warning prompt i.e., '*Warning: This notebook was not authored by Google*', select "Run anyway".

*   It is not a pre-requisite to have python programming experience to go through this example. However, for the interested learner, select "Show code" to view more details on how the platfrom has been setup.

*   After you have gone through this example, try out the task at the end of the report to enhance your learning experience. After making any changes, simply select "Runtime->Run all" to see the effect of your changes at each instant.



## Preparing the platform to execute the PyEPLAN software

When using Colab to run the PyEPLAN software, you will need to set up the eplatform using the commands defined below. 

In [35]:
#@markdown These commands make sure to remove any files remaining from previous executions.

!rm -rf sample_data
!rm -rf 1_bus

In [36]:
#@markdown  The following command installs SVN and copies the project files from Github to Colab for execution.

!apt-get -qq -y install subversion                                     &> /dev/null
!svn export https://github.com/SPS-L/pyeplan/trunk/examples/1_bus      &> /dev/null

In [37]:
#@markdown The following command installs the optimization solver that Pyeplan will utilise to solve the planning problem.

!apt-get -y -qq install glpk-utils                    &> /dev/null

In [38]:
#@markdown The following command installs PyEPLAN.

!pip install -q pyeplan==0.5.0                       &> /dev/null

## Defining the system data

Input files containing the system data are added to PyEPLAN using csv files. These files contain the load demand for each planning hour and load point, candidate and existing generators, network layout data and load and renewable generation profiles. 

The input data and templates for the format of different csv files accepted by PyEPLAN software is defined in the software [user guide](https://pyeplan.sps-lab.org/user_guide/input.html#).  

Note that as a basis, the user should to define the load demand and at least one generation input to be able to utilise the investment and operation modules of PyEPLAN.


### Defining the total electrical load consumption 

In [39]:
#@title Total power consumption

import pandas as pd
#pd.read_csv("1_bus/prep_dist.csv")

In [40]:
#@title 
#@title  { run: "auto", form-width: "50%" }

#@markdown Slide to vary the maximum consumption of the household
# reading the csv file
df = pd.read_csv("1_bus/prep_dist.csv")

maximum_load = 90 #@param {type:"slider", min:0, max:150, step:1}
  
# updating the column value/data
df.loc[0, '0'] = maximum_load
  
# writing into the file
df.to_csv("1_bus/prep_dist.csv", index=False)


### Defining the potential investment candidates

In this example, the candidate investment units have been pre-defined as shown below. The explicit definition of the different column titles can be found in the [user guide](https://pyeplan.sps-lab.org/user_guide/input.html#).  

In [41]:
#@title Wind turbines

pd.read_csv("1_bus/cwin_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,186,0,0,0,50,0,0
1,0,186,0,0,0,50,0,0


In [42]:
#@title Solar Photovoltaics
pd.read_csv("1_bus/csol_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax
0,0,109,0,0,0,50,0,0
1,0,109,0,0,0,50,0,0


In [43]:
#@title Diesel generators
pd.read_csv("1_bus/cgen_dist.csv")

,bus,icost,ocost,scost,pmin,pmax,qmin,qmax,hr
0,0,12,0.4,0,0,0,0,0,NaN
1,0,12,0.4,0,0,0,0,0,NaN


## Using the invesment and operation planning module from PyEPLAN 

In [44]:
#@markdown The following command provides access to all the PyEPLAN modules to be included in the current Colab platform.

import pyeplan as pye

### Initialising the PyEPLAN invesment planning module   

The module is initialised with different input parameters including following:
* The input directory for the data. The input data folder should consists of 'csv' files that contain data description of the load, newtork paramters and generation units as defined in the [user guide](https://pyeplan.sps-lab.org/user_guide/input.html#).
* ref_bus: Reference node
* dshed_cost: Demand Shedding Price
* rshed_cost: Renewable Shedding Price
* phase: Number of Phases (Default 3)
* sbase: Base Apparent Power (Default 1 kW)



In [45]:
#@markdown The following command is then used to set the different inputs based on the user's input parameters.
inosys = pye.inosys('1_bus', ref_bus = 0, dshed_cost = 100, rshed_cost = 0, phase = 3, )

## Determining the optimal investment 

The input arguments that should be provided to the investment planning module include:
* *solver* : program used to calculate the optimal solution to the optimisation problem.

* *onlyopr* : parameter set to 'True' if the user desires only to solve an operation planning problem. Otherwise if the user needs to find the optimal investment i.e., investment planning, the parameter is set to 'False'.

In [46]:
#@title  { vertical-output: true }
#@markdown The following command is used to solve the planning problem and determine the optimal investment candidates for the household. 

#@markdown The output 

inosys.solve(solver = 'glpk', onlyopr = False) 

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpndexxh11.glpk.raw --wglp /tmp/tmp5ykn4s3o.glpk.glp --cpxlp
 /tmp/tmpd8m3g92p.pyomo.lp
Reading problem data from '/tmp/tmpd8m3g92p.pyomo.lp'...
42 rows, 31 columns, 81 non-zeros
247 lines were read
Writing problem data to '/tmp/tmp5ykn4s3o.glpk.glp'...
210 lines were written
GLPK Simplex Optimizer, v4.65
42 rows, 31 columns, 81 non-zeros
Preprocessing...
9 rows, 14 columns, 28 non-zeros
Scaling...
 A: min|aij| =  3.333e-01  max|aij| =  4.000e+01  ratio =  1.200e+02
GM: min|aij| =  7.598e-01  max|aij| =  1.316e+00  ratio =  1.732e+00
EQ: min|aij| =  5.774e-01  max|aij| =  1.000e+00  ratio =  1.732e+00
Constructing initial basis...
Size of triangular part is 9
*     0: obj =   3.285000000e+06 inf =   0.000e+00 (4)
*     6: obj =   1.400000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (56560 bytes)
Writing basic solution to '/tmp/tmpndexxh11.glp

## Results

Below we see the optimal solution obtained to satisfy the electrical load consumption of the household. The results include the capital costs and operational costs that will be incured as well as the size and type of power units required.

 

In [47]:
#@title Total Investment and Operational Costs
inosys.resCost()

,total costs,14000.0
0,total investment costs,14000.0
1,total operation costs,0.0


In [48]:
#@title Number and capacity of wind units installed
#@markdown The capacity and location of wind units installed is:
inosys.resWind()

,Installed Capacity (kW),Bus
Unit,,
1,16.5,0
2,0.0,0


In [49]:
#@title Number and capacity of solar units installed
#@markdown The capacity and location of solar units installed is:

inosys.resSolar()

,Installed Capacity (kW),Bus
Unit,,
1,50.0,0
2,50.0,0


In [50]:
#@title Number and capacity of diesel units installed
#@markdown The capacity and location of diesel units installed is:


inosys.resConv()

,Installed Capacity (kW),Bus
Unit,,
1,0.0,0
2,0.0,0


## Your Task
In this task, you will vary the electrical load consumption using the slider under the section 'Total power consumption'. Observe the changes in the solution provided by PyEPLAN with every variation. 

After every change you make, select "Runtime->Run all" in the menu and accept the prompt. This will execute the entire notebook with the new values. 

**Task** :  Increase or decrease the maximum load consumption using the slider and investigate the changes in the optimal solution i.e.:


1.   How do the total system costs change with a change in load consumption?
2.   How does the investment solution i.e., type and size of unit installed, change with a change in load consumption?

